### Algorithm to Approximate Stabilizer Rank
* https://arxiv.org/abs/1711.07848 - On the Geometry of Stabilizer States
* $\underline{\text{First Approach - Approximating an arbitrary quantum state with a single stabilizer state (Section 4.1)}}$
    * Outline of approach:
        * Begin with arbitrary state - 
        $$\ket{\psi} = \sum_{i = 1}^{k} \alpha_i \ket{s_i}$$
        * Start at stabilizer state $\ket{s_m}$ such that it has largest $|\alpha_i|$
        * At each iteration, evaluate $N(\ket{s_m})$ (nearest neighbors of $\ket{s_m}$) by computing 
        $$\max_{\ket{\phi} \in N(\ket{s_m})} |\bra{\phi}\ket{\psi}|$$
* $\underline{\text{Second Approach - }}$

In [98]:
import stim
import numpy as np
from typing import List, Tuple

In [146]:
%run -i NonCliff_1Meas.ipynb 

[(-1+0j)Z_]
The minimum eigenvalue for this pauli sum is: (-1+0j)
The associated eigenvector is: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [147]:
def state_max_coeff(eig_vec: np.array) -> np.array:
    """ 
    Finds the constituent stabilizer state (computational basis state) with 
    largest coefficient amplitude

    Parameters:
    -----------
    eig_vec - Eigenvector 

    Returns:
    --------
    State with largest coefficient amplitude
    """
    ret_vec = np.zeros((len(eig_vec),), dtype=complex)
    max_pos = np.argmax(np.abs(eig_vec))
    ret_vec[max_pos] = 1+0j
    return ret_vec

def comp_base_vecs(num_qubits: int) -> List:
    """
    Return all computational basis vectors for a given number of qubits

    Parameters:
    -----------
    num_qubits - Number of qubits

    Returns:
    --------
    Basis vectors in list
    """
    basis_vecs = []
    for i in range(2**num_qubits):
        start_vec = np.zeros((2**num_qubits,), dtype=complex)
        start_vec[i] = 1+0j
        basis_vecs.append(start_vec)
    
    return basis_vecs

def find_nearest_neighbors(stab_state: np.array, basis_vecs: List) -> List:
    """ 
    Find and return all nearest neighbor states to chosen stabilizer state 'stab_state'

    Parameters:
    -----------
    stab_state - Stabilizer state for which we wish to find nearest neighbors
    basis_vecs - Computational Basis vectors

    Returns:
    --------
    List of nearest neighbor states
    """
    phases = [1, -1, 1j, -1j]
    norm_factor = np.sqrt(2)
    nearest_neighbors = []
    for p in phases:
        for vec in basis_vecs:
            #if (np.array_equal(stab_state, vec)):
            #    continue
            #else:
            neighbor_state = np.add(stab_state, p * vec)/norm_factor 
            nearest_neighbors.append(neighbor_state)
    
    return nearest_neighbors

def optimal_nearest_neighbor(eig_vec: np.array, nearest_neighbors: List) -> np.array:
    """ 
    Find nearest neighbor stabilizer state with maximum inner product

    Parameters:
    -----------
    eig_vec - Eigenvector we are trying to approximate

    Returns:
    --------
    Single stabilizer state that is closest to 'eig_vec'
    """
    return nearest_neighbors[np.argmax([np.abs(np.vdot(eig_vec, neighbor)) for neighbor in nearest_neighbors])]
    

In [148]:
fin_stab_approx = np.zeros((2**total_qubits,), dtype=complex)
min_eigvec_cp = min_eigvec.copy()
stab_state = state_max_coeff(min_eigvec)
basis_vecs = comp_base_vecs(total_qubits)
neighbor_list = find_nearest_neighbors(stab_state, basis_vecs)
optimal_neighbor = optimal_nearest_neighbor(min_eigvec, neighbor_list)
print(optimal_neighbor)
print(np.abs(np.vdot(optimal_neighbor, min_eigvec)))



[0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
0.7071067811865475


In [149]:
def stab_rank_approx_one(num_qubits: int, eig_vec: np.array)  -> int:
    """ 
    Approximate the stabilizer rank (using the first approach)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0
    basis_vecs = comp_base_vecs(num_qubits) # Basis vectors given number of qubits
    init_state = eig_vec.copy() # Make a copy of the original eigenvector
    final_stab_approx = np.zeros((2**num_qubits,), dtype=complex) # Keep track of the final approximation
    for _ in range(2**num_qubits):
        print("The altered eigenvector looks like: " + str(eig_vec))

        stab_state = state_max_coeff(eig_vec) # Figure out which stabilizer state to find neighbors for first
        neighbors = find_nearest_neighbors(stab_state, basis_vecs) # Find all nearest neighbors of 'stab_state'
        optimal_neighbor = optimal_nearest_neighbor(eig_vec, neighbors) # Find optimal nearest neighbor of 'stab_state'
        print("The optimal neighbor for the current state is: " + str(optimal_neighbor))

        # Subtract optimal neighbor component from eigenvector
        # Then normalize state
        eig_vec = np.subtract(eig_vec, optimal_neighbor)
        eig_vec = eig_vec/np.linalg.norm(eig_vec)

        # Add to optimal neighbor component to 'final_stab_approx'
        # Then normalize state 
        final_stab_approx = np.add(final_stab_approx, optimal_neighbor)
        final_stab_approx = final_stab_approx/np.linalg.norm(final_stab_approx)
        print("The approximated stabilizer state so far is: " + str(final_stab_approx))

        # Check to see how inner product between 'final_stab_approx' and 
        # 'eig_vec' progresses 
        print("The inner product between approximation and actual is: " + str(np.abs(np.vdot(final_stab_approx, init_state))))
        print("\n")
        approx_stab_rank += 1

    return approx_stab_rank

In [150]:
approx_stab_rank = stab_rank_approx_one(total_qubits, min_eigvec)

The altered eigenvector looks like: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
The optimal neighbor for the current state is: [0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
The approximated stabilizer state so far is: [0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
The inner product between approximation and actual is: 0.7071067811865476


The altered eigenvector looks like: [ 0.38268343+0.j -0.92387953+0.j  0.        +0.j  0.        +0.j]
The optimal neighbor for the current state is: [-0.70710678+0.j  0.70710678+0.j  0.        +0.j  0.        +0.j]
The approximated stabilizer state so far is: [7.85046229e-17+0.j 1.00000000e+00+0.j 0.00000000e+00+0.j
 0.00000000e+00+0.j]
The inner product between approximation and actual is: 7.850462293418876e-17


The altered eigenvector looks like: [ 0.55557023+0.j -0.83146961+0.j  0.        +0.j  0.        +0.j]
The optimal neighbor for the current state is: [-0.70710678+0.j  0.70710678+0.j  0.        +0.j  0.        +0.j]
The appro